# Regression Exercise - Solutions

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [125]:
import pandas as pd

In [126]:
housing = pd.read_csv('cal_housing_clean.csv')

In [127]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [128]:
housing.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [129]:
x_data = housing.drop(['medianHouseValue'],axis=1)

In [130]:
y_val = housing['medianHouseValue']

In [131]:
from sklearn.model_selection import train_test_split

In [132]:
X_train, X_test, y_train, y_test = train_test_split(x_data,y_val,test_size=0.3,random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [133]:
from sklearn.preprocessing import MinMaxScaler

In [134]:
scaler = MinMaxScaler()

In [135]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [136]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)

In [137]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [138]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [139]:
import tensorflow as tf

In [140]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [141]:
feat_cols = [ age,rooms,bedrooms,pop,households,income]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [142]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train ,batch_size=10,num_epochs=1000,
                                            shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [143]:
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_session_config': None, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_model_dir': 'C:\\Users\\Marcial\\AppData\\Local\\Temp\\tmp244d0d44', '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_tf_random_seed': 1, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [144]:
model.train(input_fn=input_func,steps=25000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Marcial\AppData\Local\Temp\tmp244d0d44\model.ckpt.
INFO:tensorflow:step = 1, loss = 5.68896e+11
INFO:tensorflow:global_step/sec: 581.528
INFO:tensorflow:step = 101, loss = 5.52951e+11 (0.172 sec)
INFO:tensorflow:global_step/sec: 611.865
INFO:tensorflow:step = 201, loss = 5.91057e+11 (0.163 sec)
INFO:tensorflow:global_step/sec: 619.467
INFO:tensorflow:step = 301, loss = 5.48503e+11 (0.162 sec)
INFO:tensorflow:global_step/sec: 683.11
INFO:tensorflow:step = 401, loss = 1.96651e+11 (0.146 sec)
INFO:tensorflow:global_step/sec: 671.59
INFO:tensorflow:step = 501, loss = 3.18206e+11 (0.149 sec)
INFO:tensorflow:global_step/sec: 680.779
INFO:tensorflow:step = 601, loss = 7.34722e+11 (0.147 sec)
INFO:tensorflow:global_step/sec: 683.111
INFO:tensorflow:step = 701, loss = 6.52219e+11 (0.146 sec)
INFO:tensorflow:global_step/sec: 641.376
INFO:tensorflow:step = 801, loss = 1.67628e+11 (0.155 sec)
INFO:te

INFO:tensorflow:global_step/sec: 619.468
INFO:tensorflow:step = 8101, loss = 6.23952e+10 (0.161 sec)
INFO:tensorflow:global_step/sec: 676.164
INFO:tensorflow:step = 8201, loss = 1.45558e+11 (0.148 sec)
INFO:tensorflow:global_step/sec: 667.117
INFO:tensorflow:step = 8301, loss = 5.34845e+10 (0.149 sec)
INFO:tensorflow:global_step/sec: 687.821
INFO:tensorflow:step = 8401, loss = 8.33874e+10 (0.145 sec)
INFO:tensorflow:global_step/sec: 673.836
INFO:tensorflow:step = 8501, loss = 1.16478e+11 (0.149 sec)
INFO:tensorflow:global_step/sec: 683.109
INFO:tensorflow:step = 8601, loss = 1.72198e+11 (0.146 sec)
INFO:tensorflow:global_step/sec: 676.163
INFO:tensorflow:step = 8701, loss = 1.35063e+11 (0.148 sec)
INFO:tensorflow:global_step/sec: 674.01
INFO:tensorflow:step = 8801, loss = 1.63489e+11 (0.148 sec)
INFO:tensorflow:global_step/sec: 639.32
INFO:tensorflow:step = 8901, loss = 9.24273e+10 (0.156 sec)
INFO:tensorflow:global_step/sec: 637.281
INFO:tensorflow:step = 9001, loss = 1.02409e+11 (0.1

INFO:tensorflow:global_step/sec: 660.481
INFO:tensorflow:step = 16201, loss = 3.05991e+10 (0.151 sec)
INFO:tensorflow:global_step/sec: 685.456
INFO:tensorflow:step = 16301, loss = 1.88008e+11 (0.146 sec)
INFO:tensorflow:global_step/sec: 683.115
INFO:tensorflow:step = 16401, loss = 1.83709e+11 (0.146 sec)
INFO:tensorflow:global_step/sec: 676.163
INFO:tensorflow:step = 16501, loss = 1.30615e+11 (0.148 sec)
INFO:tensorflow:global_step/sec: 676.163
INFO:tensorflow:step = 16601, loss = 1.21363e+11 (0.148 sec)
INFO:tensorflow:global_step/sec: 662.686
INFO:tensorflow:step = 16701, loss = 5.79005e+10 (0.150 sec)
INFO:tensorflow:global_step/sec: 676.164
INFO:tensorflow:step = 16801, loss = 2.05555e+11 (0.148 sec)
INFO:tensorflow:global_step/sec: 637.28
INFO:tensorflow:step = 16901, loss = 1.13958e+11 (0.157 sec)
INFO:tensorflow:global_step/sec: 656.145
INFO:tensorflow:step = 17001, loss = 1.56865e+11 (0.152 sec)
INFO:tensorflow:global_step/sec: 678.463
INFO:tensorflow:step = 17101, loss = 1.035

INFO:tensorflow:global_step/sec: 651.856
INFO:tensorflow:step = 24301, loss = 7.52043e+10 (0.153 sec)
INFO:tensorflow:global_step/sec: 676.164
INFO:tensorflow:step = 24401, loss = 1.08424e+11 (0.147 sec)
INFO:tensorflow:global_step/sec: 669.356
INFO:tensorflow:step = 24501, loss = 9.23162e+10 (0.150 sec)
INFO:tensorflow:global_step/sec: 680.779
INFO:tensorflow:step = 24601, loss = 5.40436e+10 (0.147 sec)
INFO:tensorflow:global_step/sec: 683.097
INFO:tensorflow:step = 24701, loss = 8.55543e+10 (0.146 sec)
INFO:tensorflow:global_step/sec: 653.971
INFO:tensorflow:step = 24801, loss = 1.211e+11 (0.153 sec)
INFO:tensorflow:global_step/sec: 629.241
INFO:tensorflow:step = 24901, loss = 6.22166e+10 (0.159 sec)
INFO:tensorflow:Saving checkpoints for 25000 into C:\Users\Marcial\AppData\Local\Temp\tmp244d0d44\model.ckpt.
INFO:tensorflow:Loss for final step: 8.48849e+10.


** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [153]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [154]:
pred_gen = model.predict(predict_input_func)

In [155]:
predictions = list(pred_gen)

INFO:tensorflow:Restoring parameters from C:\Users\Marcial\AppData\Local\Temp\tmp244d0d44\model.ckpt-25000


** Calculate the RMSE. Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [156]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [157]:
from sklearn.metrics import mean_squared_error

In [158]:
mean_squared_error(y_test,final_preds)**0.5

97921.93181985477

# Great Job!